In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 13.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of fsspec[http] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 k

In [2]:
# 3. Load Unsloth and prepare environment
from unsloth import FastLanguageModel
import torch
import os

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: We'll be using `/tmp/unsloth_compiled_cache` for temporary Unsloth patches.
Standard import failed for UnslothCPOTrainer: No module named 'UnslothCPOTrainer'. Using tempfile instead!


In [3]:
import os

path = "./checkpoint-6876/adapter_model.safetensors"
print(f"File size: {os.path.getsize(path) / (1024 * 1024):.2f} MB")


File size: 160.06 MB


In [4]:
# 4. Load Mistral 7B model with LoRA
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./checkpoint-6876",  # Unsloth's quantized Mistral 7B
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.3.19: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.138 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="./mistral_graph_training_natural_output.jsonl", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
#5. Format dataset
# Format into <s>[INST] prompt [/INST] output</s>

def formatting_func(example):
    prompt = f"<s>[INST] {example['instruction']} [/INST] {example['output']}</s>"
    
    return {"text": prompt}
 
dataset = dataset.map(formatting_func, remove_columns=dataset.column_names)

Map:   0%|          | 0/510 [00:00<?, ? examples/s]

In [7]:
# Tokenize and save
dataset = dataset.map(
    lambda samples: tokenizer(samples["text"], truncation=True, padding=False, max_length=2048),
    batched=True,
    num_proc=4,
)
dataset.save_to_disk("tokenized_parsed_final")
print("✅ Tokenized dataset saved!")

Map (num_proc=4):   0%|          | 0/510 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/510 [00:00<?, ? examples/s]

✅ Tokenized dataset saved!


In [8]:
# Set format for Trainer compatibility
dataset.set_format(type="torch")

In [9]:
# ✅ 7. Load Unsloth and environment setup 
from unsloth import is_bfloat16_supported
from transformers import TrainingArguments
from trl import SFTTrainer

In [11]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    tokenizer = tokenizer,
    dataset_text_field = "text",
    max_seq_length = 2048,
    packing = False,
    args = TrainingArguments(
        output_dir = "./mistral-prosense-parsed",
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        num_train_epochs = 4,
        learning_rate = 2e-4,
        fp16 = True,
        bf16 = False,  # use hardware-accurate setting
        logging_steps = 1,
        save_strategy="steps",
        save_steps=1000,
        save_total_limit=2,
        resume_from_checkpoint=True,  # ✅ this will resume from the checkpoint
        gradient_checkpointing = True,
        optim = "adamw_8bit",
        lr_scheduler_type = "cosine",
    ),
)


In [12]:
# ✅ 9. Train
trainer.train()

# ✅ 10. Save model and tokenizer
model.save_pretrained("./mistral-finetuning-parseddata")
tokenizer.save_pretrained("./mistral-finetuning-parseddata")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 510 | Num Epochs = 4 | Total steps = 64
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 2 x 1) = 32
 "-____-"     Trainable parameters = 41,943,040/7,000,000,000 (0.60% trained)
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.911600
2,2.803400
3,2.918500
4,2.271800
5,1.915300
6,1.699100
7,1.608500
8,1.466200
9,1.393800
10,1.394300


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or

('./mistral-finetuning-parseddata/tokenizer_config.json',
 './mistral-finetuning-parseddata/special_tokens_map.json',
 './mistral-finetuning-parseddata/tokenizer.model',
 './mistral-finetuning-parseddata/added_tokens.json',
 './mistral-finetuning-parseddata/tokenizer.json')